In [1]:
import numpy as np
from ADAL import ADAL
from IRWA import IRWA

In [9]:
from qpsolvers import solve_qp

M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
P = M.T @ M  # this is a positive definite matrix
q = np.array([3.0, 2.0, 3.0]) @ M
G = np.array([[1.0, 2.0, 1.0], [2.0, 0.0, 1.0], [-1.0, 2.0, -1.0]])
h = np.array([3.0, 2.0, -2.0])
A = np.array([[1.0, 1.0, 1.0]])
b = np.array([1.0])

x = solve_qp(P, q, G, h, A, b, solver="osqp")
print(f"QP solution: {x = }")

D:\software\python3.11\Lib\site-packages\qpsolvers\solvers\__init__.py:752: UserWarning: no QP solver found on your system, you can install solvers from PyPI by ``pip install qpsolvers[open_source_solvers]``
  warnings.warn(


SolverNotFound: found solvers [] but 'cvsopt' is not one of them; if 'cvsopt' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can run ``pip install qpsolvers[cvsopt]``

In [10]:
x = ADAL(A,G,-b,-h,q,P,np.array([1.0, 1.0, 1.0]),np.array([1.0, 1.0, 1.0,1.0]),np.array([1.0, 1.0, 1.,1.0]),1,1,1)
print(x)

[ 0.30769214 -0.69230883  1.38461612]


In [16]:
x = IRWA(A,G,-b,-h,q,P,np.array([1,1,1]),epsilon0=1e-100*np.array([1.0, 1.0, 1.0,1.0]),eta= 0.7,gamma=1/6,M= 10000,sigma=1e-6,sigma_prime=1e-8)
print(x)


LinAlgError: Singular matrix